In [1]:
import os
import pandas as pd
from PIL import Image

import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
test_dir_glob = glob.glob(test_dir + 'images/*')

train_dir = '/opt/ml/input/data/train'
train_dir_glob = glob.glob(train_dir + '/images/**/*')

test_csv = '/opt/ml/input/data/eval/info.csv'
train_csv = '/opt/ml/input/data/train/train1.csv'

count = 0

for files in test_dir_glob:
    if count == 10:
        break

    if files[-3:] == 'jpg':
        print(files)
        count += 1

In [28]:
train_df = pd.read_csv(train_csv)
train_df.drop(columns='Unnamed: 0',inplace=True)
train_df

,id,gender,race,age,mask,path
0,000001,female,Asian,45,0,000001_female_Asian_45/normal.jpg
1,000001,female,Asian,45,1,000001_female_Asian_45/mask1.jpg
2,000001,female,Asian,45,1,000001_female_Asian_45/mask2.jpg
3,000001,female,Asian,45,1,000001_female_Asian_45/mask3.jpg
4,000001,female,Asian,45,1,000001_female_Asian_45/mask4.jpg
...,...,...,...,...,...,...
18895,006959,male,Asian,19,1,006959_male_Asian_19/mask2.jpg
18896,006959,male,Asian,19,1,006959_male_Asian_19/mask3.jpg
18897,006959,male,Asian,19,1,006959_male_Asian_19/mask4.jpg
18898,006959,male,Asian,19,1,006959_male_Asian_19/mask5.jpg


In [29]:
train_df['class'] = 19

train_df.loc[train_df[(train_df['age'] < 30) & (train_df['gender'] =='male') & (train_df['mask'] ==1)].index,'class'] = 0

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60) & (train_df['gender'] =='male') & (train_df['mask'] ==1)].index,'class'] = 1

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='male') & (train_df['mask'] ==1)].index,'class'] = 2

train_df.loc[train_df[(train_df['age'] < 30) & (train_df['gender'] =='female') & (train_df['mask'] ==1)].index,'class'] = 3

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60)& (train_df['gender'] =='female') & (train_df['mask'] ==1)].index,'class'] = 4

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='female') & (train_df['mask'] ==1)].index,'class'] = 5

train_df.loc[train_df[(train_df['age'] < 30) & (train_df['gender'] =='male') & (train_df['mask'] ==2)].index,'class'] = 6

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60) & (train_df['gender'] =='male') & (train_df['mask'] ==2)].index,'class'] = 7

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='male') & (train_df['mask'] ==2)].index,'class'] = 8

train_df.loc[train_df[(train_df['age'] < 30 ) & (train_df['gender'] =='female') & (train_df['mask'] ==2)].index,'class'] = 9

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60) & (train_df['gender'] =='female') & (train_df['mask'] ==2)].index,'class'] = 10

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='female') & (train_df['mask'] ==2)].index,'class'] = 11

train_df.loc[train_df[(train_df['age'] < 30) & (train_df['gender'] =='male') & (train_df['mask'] ==0)].index,'class'] = 12

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60) & (train_df['gender'] =='male') & (train_df['mask'] ==0)].index,'class'] = 13

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='male') & (train_df['mask'] ==0)].index,'class'] = 14

train_df.loc[train_df[(train_df['age'] < 30) & (train_df['gender'] =='female') & (train_df['mask'] ==0)].index,'class'] = 15

train_df.loc[train_df[(train_df['age'] >= 30)& (train_df['age'] <60) & (train_df['gender'] =='female') & (train_df['mask'] ==0)].index,'class'] = 16

train_df.loc[train_df[(train_df['age'] >= 60) & (train_df['gender'] =='female') & (train_df['mask'] ==0)].index,'class'] = 17

In [30]:
train_df

,id,gender,race,age,mask,path,class
0,000001,female,Asian,45,0,000001_female_Asian_45/normal.jpg,16
1,000001,female,Asian,45,1,000001_female_Asian_45/mask1.jpg,4
2,000001,female,Asian,45,1,000001_female_Asian_45/mask2.jpg,4
3,000001,female,Asian,45,1,000001_female_Asian_45/mask3.jpg,4
4,000001,female,Asian,45,1,000001_female_Asian_45/mask4.jpg,4
...,...,...,...,...,...,...,...
18895,006959,male,Asian,19,1,006959_male_Asian_19/mask2.jpg,0
18896,006959,male,Asian,19,1,006959_male_Asian_19/mask3.jpg,0
18897,006959,male,Asian,19,1,006959_male_Asian_19/mask4.jpg,0
18898,006959,male,Asian,19,1,006959_male_Asian_19/mask5.jpg,0


In [15]:
import torchvision.models as models
MyModel = models.resnet50(pretrained=True)

In [16]:
MyModel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [32]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=8).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

TypeError: forward() got an unexpected keyword argument 'num_classes'